In [1]:
import os
import json
import numpy as np
import sqlite3
import matplotlib.pyplot as plt

In [2]:
### Configuration

SAVE_FIGURES = True
SQLITE_PATH = "/eos/user/m/mantydze/DQMIO2017/ZeroBias2017D.sqlite"

RUN = 302040
HISTOGRAM = "RPC/AllHits/SummaryHistograms/Occupancy_for_Barrel"

In [3]:
### Create export directory RUN/HISTOGRAM/

fig_name = HISTOGRAM.replace("/", ".")

# Run directory
if not os.path.exists(str(RUN)):
    os.makedirs(str(RUN))

# histogram directory
dir_name = os.path.join(str(RUN), fig_name)
if not os.path.exists(dir_name):
    os.makedirs(dir_name)

In [4]:
conn = sqlite3.connect(SQLITE_PATH)
conn.row_factory = sqlite3.Row
c = conn.cursor()

In [5]:
def create_heatmap(width, height, data, title="", filename=None):
    
    heatmap = np.array(data).reshape(height, width)
    
    # remove border
    heatmap = heatmap[1:height-1, 1:width-1]
    
    plt.figure(figsize=(15,10))
    plt.title(title)
    plt.colorbar(plt.pcolor(heatmap, cmap="rainbow"))
    
    # Annotate
    for y in range(heatmap.shape[0]):
        for x in range(heatmap.shape[1]):
            plt.text(x + 0.5, y + 0.5, '%d' % heatmap[y, x],
                    horizontalalignment='center',
                    verticalalignment='center',)
    
    if SAVE_FIGURES:
        plt.savefig(filename)    
        plt.close()

In [7]:
query = c.execute("Select * FROM monitorelements WHERE fromrun = ? and name = ?", (RUN, HISTOGRAM))

rows = query.fetchall()

row = dict(rows[0])
jvalue = json.loads((row["value"]))

width = jvalue["fXaxis"]["fNbins"] + 2 # +2 because border is added on both X and Y axis
height = jvalue["fYaxis"]["fNbins"] + 2
title = jvalue["fTitle"]
total = np.zeros(width*height) # Accumulated values during whole run

# Cross-check if numbers match
if jvalue["fNcells"] != (width*height):
    import sys
    sys.exit()

# Iterate over all lumisections
for row in rows:
    
    row = dict(row)
    ls = row["fromlumi"]
    print(ls)
    jvalue = json.loads((row["value"]))
    data = jvalue["fArray"]
    
    _filename = os.path.join(dir_name, "{title}-{run}-{ls:04d}.png".format(title=title, run=RUN, ls=ls))
    _title = "{title} LS #{ls}".format(title=title, ls=ls)

    # 2D histogram                                 
    if jvalue["_typename"] == "TH2F":
        create_heatmap(width, height, data, title=_title, filename=_filename)
        total = np.add(total, data)                     

# 2D histogram        
if jvalue["_typename"] == "TH2F":
    _filename = os.path.join(dir_name, "{title}-{run}.png".format(title=fig_name, run=RUN))
    create_heatmap(width, height, total, title=title, filename=_filename)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
